In [14]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI


import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()

# Set environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [3]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./Reasearch-Papers")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    
final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'Reasearch-Papers/The quality management ecosystem for predictive maintenance in the Industry 4.0 era_.pdf', 'page': 0}, page_content='THEORETICAL ARTICLE Open Access\nThe quality management ecosystem for\npredictive maintenance in the Industry\n4.0 era\nSang M. Lee1, DonHee Lee2*and Youn Sung Kim2\n* Correspondence: dhlee04@inha.ac.\nkr\n2College of Business Administration,\nInha University, Incheon, South\nKorea\nFull list of author information is\navailable at the end of the articleAbstract\nThe Industry 4.0 era requires new quality management systems due to the ever\nincreasing complexity of the global business environment and the advent of advanced\ndigital technologies. This study presents new ideas for predictive quality management\nbased on an extensive review of the literature on quality management and five real-\nworld cases of predictive quality managem ent based on new technologies. The results\nof the study indicate that advanced technology ena

In [5]:
db_chroma = Chroma.from_documents(final_documents, OpenAIEmbeddings())

/home/anas/Desktop/Literature-Review-Writer/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [33]:
literature_review_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Based on the following research papers:
    {context}
    
    Write a comprehensive literature review addressing the research question:
    {question}
    
    The literature review should:
    1. Summarize key findings and methodologies
    2. Identify trends and patterns in the research
    3. Highlight gaps or contradictions in the existing literature
    4. Explain how the current research relates to previous studies
    """
)

In [34]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)




In [31]:
llm_chain = LLMChain(llm=llm, prompt=literature_review_template)


In [35]:
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
)

In [36]:
retriever = db_chroma.as_retriever(search_kwargs={"k": 5})

In [37]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": literature_review_template
    }
)

In [39]:
def generate_literature_review(research_question):
    # Retrieve relevant documents
    docs = retriever.get_relevant_documents(research_question)
    
    # Use StuffDocumentsChain to process the documents and generate the review
    result = stuff_chain.run(input_documents=docs, question=research_question)
    
    return result

In [42]:
research_question = "Tell me about these research papers and how they relate to bearing?"
result = generate_literature_review(research_question)
print(result)

The research papers reviewed in this literature review focus on the condition monitoring and fault diagnosis of bearings, particularly in induction motors. The studies utilize various techniques such as artificial neural networks (ANN), feature extraction, classification, and deep learning algorithms to enhance the detection and analysis of bearing faults. The advancements in these methodologies have led to more efficient and accurate diagnosis of bearing faults, which are crucial in preventing machinery breakdowns and improving overall performance.

Patel and Giri (2016) conducted a study on the performance evaluation of Bearing Damage Index (BDI) using ANN for condition monitoring of induction motor bearings. They found that the ANN-based approach provided effective results in detecting bearing faults. In their subsequent study in 2018, Patel and Giri focused on feature extraction and classification techniques for bearing fault analysis in induction motors, demonstrating the importan